In [ ]:
### MDP Value Iteration and Policy Iteration

import numpy as np

np.set_printoptions(precision=3)

def bellman_backup(state, action, R, T, gamma, V):
    """
    Perform a single Bellman backup.

    Parameters
    ----------
    state: int
    action: int
    R: np.array (num_states, num_actions)
    T: np.array (num_states, num_actions, num_states)
    gamma: float
    V: np.array (num_states)

    Returns
    -------
    backup_val: float
    """
    backup_val = 0.
    ############################
    # YOUR IMPLEMENTATION HERE #
    backup_val = R[state,action]+gamma*np.dot(T[state,action],V)
    ############################

    return backup_val

def policy_evaluation(policy, R, T, gamma, tol=1e-3):
    """
    Compute the value function induced by a given policy for the input MDP
    Parameters
    ----------
    policy: np.array (num_states)
    R: np.array (num_states, num_actions)
    T: np.array (num_states, num_actions, num_states)
    gamma: float
    tol: float

    Returns
    -------
    value_function: np.array (num_states)
    """
    num_states, num_actions = R.shape
    value_function = np.zeros(num_states)

    ############################
    # YOUR IMPLEMENTATION HERE #
    while True:
      nxt_value_function=np.zeros(num_states)
      stop=True
      for state in range(num_states):
        nxt_value_function[state] = R[state,policy[state]]+gamma*np.dot(T[state,policy[state]],value_function)
        if(abs(nxt_value_function[state]-value_function[state])>tol):
          stop=False
      if(stop):
        break
      value_function=nxt_value_function


    ############################
    return value_function


def policy_improvement(policy, R, T, V_policy, gamma):
    """
    Given the value function induced by a given policy, perform policy improvement
    Parameters
    ----------
    policy: np.array (num_states)
    R: np.array (num_states, num_actions)
    T: np.array (num_states, num_actions, num_states)
    V_policy: np.array (num_states)
    gamma: float

    Returns
    -------
    new_policy: np.array (num_states)
    """
    num_states, num_actions = R.shape
    new_policy = np.zeros(num_states, dtype=int)

    ############################
    # YOUR IMPLEMENTATION HERE #
    for state in range(num_states):
      mx=0.0
      bst_action=0
      for action in range(num_actions):
        val=bellman_backup(state,action,R,T,gamma,V_policy)
        if(val>mx):
          bst_action=action
          mx=val
      new_policy[state]=bst_action
    ############################
    return new_policy


def policy_iteration(R, T, gamma, tol=1e-3):
    """Runs policy iteration.

    You should call the policy_evaluation() and policy_improvement() methods to
    implement this method.
    Parameters
    ----------
    R: np.array (num_states, num_actions)
    T: np.array (num_states, num_actions, num_states)

    Returns
    -------
    V_policy: np.array (num_states)
    policy: np.array (num_states)
    """
    num_states, num_actions = R.shape
    V_policy = np.zeros(num_states)
    policy = np.zeros(num_states, dtype=int)
    ############################
    # YOUR IMPLEMENTATION HERE #
    while True:
      V_upd=policy_evaluation(policy, R, T, gamma, tol)
      stop=True
      for state in range(num_states):
        if(abs(V_upd[state]-V_policy[state])>tol):
          stop=False
      if(stop):
        break;
      V_policy=V_upd
      policy=policy_improvement(policy, R, T, V_policy, gamma)

    ############################
    return V_policy, policy


def value_iteration(R, T, gamma, tol=1e-3):
    """Runs value iteration.
    Parameters
    ----------
    R: np.array (num_states, num_actions)
    T: np.array (num_states, num_actions, num_states)

    Returns
    -------
    value_function: np.array (num_states)
    policy: np.array (num_states)
    """
    num_states, num_actions = R.shape
    value_function = np.zeros(num_states)
    policy = np.zeros(num_states, dtype=int)
    ############################
    # YOUR IMPLEMENTATION HERE #
    while True:
      stop=True
      nxt_value_function=np.zeros(num_states)
      for state in range(num_states):
        mx=0.0
        bst_action=0
        for action in range(num_actions):
          val=bellman_backup(state, action, R, T, gamma, value_function)
          if(val>mx):
            mx=val
            bst_action=action
        if(abs(mx-value_function[state])>tol):
          stop=False
        nxt_value_function[state]=mx
        policy[state]=bst_action
      if(stop):
        break
      value_function=nxt_value_function
    ############################
    return value_function, policy


# Edit below to run policy and value iteration on different configurations
# You may change the parameters in the functions below
SEED = 1234

RIVER_CURRENT = 'STRONG'
assert RIVER_CURRENT in ['WEAK', 'MEDIUM', 'STRONG']
env = RiverSwim(RIVER_CURRENT, SEED)

R, T = env.get_model()
discount_factor = 0.94

print("\n" + "-" * 25 + "\nBeginning Policy Iteration\n" + "-" * 25)

V_pi, policy_pi = policy_iteration(R, T, gamma=discount_factor, tol=1e-3)
print(V_pi)
print([['L', 'R'][a] for a in policy_pi])

print("\n" + "-" * 25 + "\nBeginning Value Iteration\n" + "-" * 25)

V_vi, policy_vi = value_iteration(R, T, gamma=discount_factor, tol=1e-3)
print(V_vi)
print([['L', 'R'][a] for a in policy_vi])



-------------------------
Beginning Policy Iteration
-------------------------
[0.075 0.089 0.169 0.424 1.17  3.3  ]
['R', 'R', 'R', 'R', 'R', 'R']

-------------------------
Beginning Value Iteration
-------------------------
[0.079 0.093 0.173 0.428 1.173 3.303]
['R', 'R', 'R', 'R', 'R', 'R']


0.67 -> largest discount factor such that an optimal agent starting
in the initial far-left state does not swim up the river for weak current.  
0.77 -> largest discount factor such that an optimal agent starting
in the initial far-left state does not swim up the river for medium current.  
0.93 -> largest discount factor such that an optimal agent starting
in the initial far-left state does not swim up the river for strong current.  

In [ ]:
import copy
import random
import numpy as np

class RiverSwim:
    def __init__(self, current, seed=1234):
        self.num_states = 6
        self.num_actions = 2  # O <=> LEFT, 1 <=> RIGHT

        # Larger current makes it harder to swim up the river
        self.currents = ['WEAK', 'MEDIUM', 'STRONG']
        assert current in self.currents
        self.current = self.currents.index(current) + 1
        assert self.current in [1, 2, 3]

        # Configure reward function
        R = np.zeros((self.num_states, self.num_actions))
        R[0, 0] = 0.005
        R[5, 1] = 1.

        # Configure transition function
        T = np.zeros((self.num_states, self.num_actions, self.num_states))

        # Encode initial and rewarding state transitions
        T[0, 0, 0] = 1.
        T[0, 1, 0] = 0.6
        T[0, 1, 1] = 0.4

        T[5, 1, 5] = 0.6
        T[5, 1, 4] = 0.4
        T[5, 0, 4] = 1.

        # Encode intermediate state transitions
        for s in range(1, self.num_states - 1):
            left, right = 0, 1

            # Going left always succeeds
            T[s, left, s - 1] = 1.

            # Going right sometimes succeeds
            T[s, right, s] = 0.6
            T[s, right, s - 1] = 0.09 * self.current
            T[s, right, s + 1] = 0.4 - T[s, right, s - 1]
            assert np.isclose(np.sum(T[s, right]), 1.)

        self.R = np.array(R)
        self.T = np.array(T)

        # Agent always starts at the opposite end of the river
        self.init_state = 0
        self.curr_state = self.init_state

        self.seed = seed
        random.seed(self.seed)
        np.random.seed(self.seed)

    def get_model(self):
        return copy.deepcopy(self.R), copy.deepcopy(self.T)

    def reset(self):
        return self.init_state

    def step(self, action):
        reward = self.R[self.curr_state, action]
        next_state = np.random.choice(range(self.num_states), p=self.T[self.curr_state, action])
        self.curr_state = next_state
        return reward, next_state